Trying to use guide https://www.kaggle.com/amirrezaeian/time-series-data-analysis-using-lstm-tutorial

In [1]:
import sys 
import numpy as np # linear algebra
# from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
# import seaborn as sns # used for plot interactive graph. 
# from sklearn.model_selection import train_test_split # to split the data into two parts
# from sklearn.cross_validation import KFold # use for cross validation
# from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
# from sklearn.pipeline import Pipeline # pipeline making
# from sklearn.model_selection import cross_val_score
# from sklearn.feature_selection import SelectFromModel
# from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score

## for Deep-learing:
from keras import backend as K
import keras
from keras.layers import Dense, Input
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
# from keras.callbacks import EarlyStopping
# from keras.utils import np_utils
import itertools
from keras.layers import LSTM
# from keras.layers.convolutional import Conv1D
# from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from keras.layers import Activation
import pickle
import os

## from REFIT tutorial
from lxml import objectify
%matplotlib inline

C:\Users\Ford\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
path=r'dataset/REFIT_BUILDING_SURVEY.xml'
tree = objectify.parse(path)
root = tree.getroot()
NS={'a':'http://www.refitsmarthomes.org'}

path=r'dataset/REFIT_TIME_SERIES_VALUES.csv'

cache = 'csv.pkl'

if os.path.isfile(cache):
    with open(cache, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        csv = pickle.load(f)
else:
    try:
        csv
    except:
        csv=pd.read_csv(path, index_col=0, parse_dates=[1])
    with open(cache, 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(csv, f, pickle.HIGHEST_PROTOCOL)
    
    
    
## Global parameters
building_num = 2

In [ ]:
buildings = list(range(1,19))
del buildings[4]
del buildings[2]
period = 'D'

data = []

## Get gas meter data
appliance    = 1 #Number of gas meter
for building_num in buildings:
    tsv = root.xpath('./a:Stock/a:Building[%d]/a:Meter[%d]/a:Sensor/a:TimeSeriesVariable' % (building_num, appliance), namespaces=NS)
    id = tsv[0].get('id')
    gasdf = csv.loc[id]
    header = 'gas_' + str(building_num)
    data_gas = {header: gasdf['data'].tolist()}
    time_gas = gasdf['dateTime'].tolist()
    gasdf = pd.DataFrame(data_gas, index=time_gas).resample(period).sum()
    data.append(gasdf)

## Merge two DataFrames gas meter and weather
df = pd.concat(data, axis=1)
df.index.name = 'time'
    
# print(times)

In [ ]:
df.isnull().sum(axis=1)

In [ ]:
## Get climate data
elements=root.xpath('./a:Stock/a:Climate/a:Sensor/a:TimeSeriesVariable', namespaces=NS)
data_clim = {}
time_clim = []
agg = {}
period = 'D'

for e in elements:
    id = e.get('id')
    v = csv.loc[id]
    sensor_name = e.get('variableType').replace(' ', '_')
    if len(time_clim) < 1:
        time_clim = v['dateTime'].tolist() 
    if sensor_name == 'Total_rainfall':
        tmp = np.sum
    else:
        tmp = np.mean
    agg[sensor_name] = tmp
    data_clim[sensor_name] = v['data'].tolist()
climdf = pd.DataFrame(data_clim, index=time_clim).resample(period).agg(agg)
##clean climdf (only when not resamplet for Day 'D')
#climdf = climdf.iloc[1::2] ## Some timeSteps are not in gas_meter indexes (15 mins and 45 mins values, 00 and 30 mins are in)


buildings = list(range(1,21))
del buildings[4]
del buildings[2]

## Get gas meter data
appliance    = 1 #Number of gas meter
tsv = root.xpath('./a:Stock/a:Building[%d]/a:Meter[%d]/a:Sensor/a:TimeSeriesVariable' % (building_num, appliance), namespaces=NS)
id = tsv[0].get('id')
gasdf = csv.loc[id]
data_gas = {'gas': gasdf['data'].tolist()}
time_gas = gasdf['dateTime'].tolist()
gasdf = pd.DataFrame(data_gas, index=time_gas).resample(period).sum()
agg['gas'] = np.sum

## Merge two DataFrames gas meter and weather
df = pd.concat([climdf, gasdf], axis=1)
df.index.name = 'time'
## clean rows
df = df[np.isfinite(df['Air_temperature'])]   # We don't need data where no weather forecast
lastind = time_gas[-1]
df = df[:lastind]                             # We don't need data where no meter values


## NaN rows detection
j = len(df.columns) - 1 # last column is 'gas'
df.iloc[:,j]=df.iloc[:,j].fillna(df.iloc[:,j].mean())
df.isnull().sum()

In [ ]:
df['Wind_speed'].argmax()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
cols = list(range(len(df.columns)))
i = 1

tmp = pd.DataFrame(df.resample('6M').mean())

for col in tmp.columns:
    tmp[col].plot(kind='bar', color='r', legend=True )
    plt.xticks(rotation=45)
    plt.show()

## convert illuminance values 

In [ ]:
period = 'W' ## M, Q, W, A == Year

## resampling over week and computing mean
df.Air_temperature.resample(period).plot(color='y', legend=True)
# df.Average_barometric_pressure.resample(period).mean().plot(color='r', legend=True)
# df.Relative_humidity.resample(period).mean().plot(color='b', legend=True)
df.Total_rainfall.resample(period).plot(color='r', legend=True)
# df.Wind_speed.resample(period).plot(color='b', legend=True)
df.gas.resample(period).plot(color='g', legend=True)
plt.show()

In [ ]:
def pltCor(df):
    print('\n'.join(['%d: %s' % (i, j) for i, j in enumerate(df.columns.tolist())]))
    # Correlations among columns
    plt.matshow(df.corr(method='spearman'),vmax=1,vmin=-1,cmap='PRGn')
    plt.colorbar()
    plt.show()

pltCor(df)

In [ ]:
# Correlations of mean of features resampled over months

print('\n'.join(['%d: %s' % (i, j) for i, j in enumerate(df.columns.tolist())]))

plt.matshow(df.resample('W').mean().corr(method='spearman'),vmax=1,vmin=-1,cmap='PRGn')
plt.title('resampled over month', size=15)
plt.colorbar()
plt.margins(0.02)
plt.matshow(df.resample('6M').mean().corr(method='spearman'),vmax=1,vmin=-1,cmap='PRGn')
plt.title('resampled over year', size=15)
plt.colorbar()
plt.show()

## Start to prepare data and build model

In [ ]:
df_resample = df.resample('D').agg(agg)
df_resample.shape

In [ ]:
kqk = df_resample.index.to_period('M').start_time.unique()[1:]
# print(kqk)

kqk = [df_resample.index.get_loc(i) for i in kqk]
kqk.insert(0,0)
kqk[-1] = df_resample.shape[0]
print('-'.join(map(str,kqk)))

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    dff = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(dff.shift(i))
        names += [(data.columns[j] + ' (t-%d)' % (i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(dff.shift(-i))
        if i == 0:
            names += [(data.columns[j] + ' (t)') for j in range(n_vars)]
        else:
            names += [(data.columns[j] + ' (t+%d)' % (i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
df_resample.columns

In [ ]:
# frame as supervised learning
reframed = series_to_supervised(df_resample, 3, 1)


# drop columns we don't want to predict
predName = 'gas (t)'
tmp = reframed.corr(method='spearman')[predName]
del tmp[predName]


#Select features with relevat correlation
corLev = 0.6
tmp = tmp.loc[tmp.abs() < corLev].index.tolist()
kek = list(map(reframed.columns.get_loc, tmp))
reframed.drop(reframed.columns[kek], axis=1, inplace=True)

pltCor(reframed)
print('Shape is: ', reframed.shape)

In [ ]:
predNamepredName = 'gas (t)'
tmp = reframed.corr(method='spearman')[predName]
del tmp[predName]

tmp
# tmp = tmp.loc[tmp.abs() > 0.5].index.tolist()

In [ ]:
## Scale data to (0..1) values range## Scale 
values = reframed.values

X,Y = values[:, :-1], values[:, -1]
scalerX = MinMaxScaler(feature_range=(0, 1))
X = scalerX.fit_transform(X)
scalerY = MinMaxScaler(feature_range=(0, 1))
Y = scalerY.fit_transform(Y.reshape(-1, 1))

In [ ]:
print(X.shape)
print(Y.shape)
assert Y.shape[0] == X.shape[0]

In [ ]:
comp = min(17, X.shape[1])

pca = PCA(n_components=comp) # PCA(n_components=2) # PCA(0.9)

X = pca.fit_transform(X=X) 

# to get how much variance was retained
print(pca.explained_variance_ratio_.sum())
print(X.shape)

In [ ]:
## Split into train and test sets
n_train_time = int(reframed.shape[0] * 0.67)
train_X, train_y = X[:n_train_time, :], Y[:n_train_time, :]
test_X, test_y   = X[n_train_time:, :], Y[n_train_time:, :]


# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) 
# We reshaped the input into the 3D format as expected by LSTMs, namely [samples, timesteps, features].

In [ ]:
import random
random.seed(42)

model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))
# model.add(Dense(100))
# model.add(Dropout(0.2))
# model.add(Activation('relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')


# fit network
history = model.fit(train_X, train_y, epochs=30, batch_size=7, validation_data=(test_X, test_y), verbose=0, shuffle=False)


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()


# make a prediction
yhat = model.predict(test_X)
# invert scaling for forecast
inv_yhat = scalerY.inverse_transform(yhat)
# invert scaling for actual
inv_y = scalerY.inverse_transform(test_y)


# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)


interval = 100
## time steps, every step is one hour (you can easily convert the time step to the actual time index)
## for a demonstration purpose, I only compare the predictions in 200 hours. 
aa=[x for x in range(interval)]
plt.plot(aa, inv_y[:interval], marker='.', label="actual")
plt.plot(aa, inv_yhat[:interval], 'r', label="prediction")
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show()

In [ ]:
pred = model.predict(X.reshape((X.shape[0], 1, X.shape[1])))

In [ ]:
data = []
time = []

yy = scalerY.inverse_transform(Y)
ypred = scalerY.inverse_transform(pred)
ypred = np.array([i if i >= 0.0 else 0.0 for i in ypred]).reshape(-1,1)

for k in range(len(kqk)-1):
    data.append(np.sqrt(mean_squared_error(yy[kqk[k]:kqk[k+1]], ypred[kqk[k]:kqk[k+1]])))
    time.append(df_resample.index[kqk[k]])
    
rmse = np.sqrt(mean_squared_error(yy, ypred))
print('Test RMSE: %.3f' % rmse)

In [ ]:
plt.plot(time, data)
plt.xticks(rotation=45)
plt.show()

In [ ]:
begin = 0
end = yy.shape[0]

aa = list(range(begin, end))
plt.figure(figsize=(20, 5))
plt.plot(aa,ypred[begin:end], marker='.', label="prediction")
plt.plot(aa,yy[begin:end],'r', marker='*', label="actual")
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show()


kak = [abs((ypred[i] - yy[i])) for i in range(begin, end)]
plt.figure(figsize=(20, 5))
plt.plot(aa, kak)
plt.show()

In [ ]:
# import shelve
# db = shelve.open("brute_PCA_dim.pkl")
# db['data'] = data
# db.close()

In [ ]:
# plt.plot(list(range(30,80,5)), data)
# plt.show()

# предсказывает отрицательные величины

# pca
# Автокорреляция
# попробовать из статейки
# predict season
# Other topology (LSTM at least)
# Добавить день недели, месяц, число, часы в инпуты

#Предсказывать потребление для района, нахуй, а не для одной семьи/дома!

In [ ]:
# ## to release gpu

# def free_gpu():
#     try:
#         del model
#     except:
#         print('No model')
#     try:
#         del history
#     except:
#         print('No history')
#     import gc
#     K.clear_session()
#     for i in range(15): gc.collect()

# free_gpu()